У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [27]:
customer_df = pd.read_csv('./customer_segmentation_train.csv')

In [19]:
customer_df.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [48]:
categorical_cols = customer_df.select_dtypes(include='object').columns.tolist()

In [50]:
for column in categorical_cols:
    customer_df.fillna({column: 'Unknown'}, inplace=True)

In [53]:
customer_df.fillna( {'Work_Experience': customer_df['Work_Experience'].median()} , inplace=True)

In [54]:
customer_df.fillna( {'Family_Size': customer_df['Family_Size'].median()} , inplace=True)

In [55]:
customer_df.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [56]:
train_df, val_df = train_test_split(customer_df, test_size=0.2, random_state=42, stratify=customer_df['Segmentation'])

In [57]:
input_cols = list(train_df.columns)[1:-1]
target_col = 'Segmentation'

train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()
val_inputs = train_df[input_cols].copy()
val_targets = train_df[target_col].copy()


numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()[:-1]

categorical_cols = train_inputs.select_dtypes(include='object').columns.tolist()

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [64]:
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler(feature_range=(0, 1)))
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [69]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
numeric_indices = [train_inputs.columns.get_loc(col) for col in numeric_cols]
categorical_indices = [train_inputs.columns.get_loc(col) for col in categorical_cols]

smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=0)

train_inputs_transformed = preprocessor.fit_transform(train_inputs)

X_train_smotenc, y_train_smotenc = smote_nc.fit_resample(train_inputs_transformed, train_targets)

smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(train_inputs_transformed, train_targets)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [72]:
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [74]:
#оригінальні дані 
log_reg = LogisticRegression(solver='liblinear')
ovr_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', OneVsRestClassifier(log_reg))
])
ovr_model.fit(train_inputs, train_targets)
ovr_predictions = ovr_model.predict(val_inputs)

print(classification_report(val_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.46      0.44      1578
           B       0.42      0.17      0.24      1486
           C       0.50      0.65      0.56      1576
           D       0.63      0.71      0.67      1814

    accuracy                           0.51      6454
   macro avg       0.49      0.50      0.48      6454
weighted avg       0.50      0.51      0.49      6454



In [80]:
from imblearn.pipeline import Pipeline as ImbPipeline

In [82]:
#SMOTENC
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTENC(categorical_features=categorical_indices, random_state=0)),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])

pipeline.fit(train_inputs, train_targets)

ovr_predictions = pipeline.predict(val_inputs)
print(classification_report(val_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.49      0.46      1578
           B       0.40      0.25      0.31      1486
           C       0.50      0.59      0.54      1576
           D       0.66      0.67      0.67      1814

    accuracy                           0.51      6454
   macro avg       0.50      0.50      0.49      6454
weighted avg       0.51      0.51      0.50      6454



In [83]:
#SMOTETomek
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTETomek(random_state=0)),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])

pipeline.fit(train_inputs, train_targets)

ovr_predictions = pipeline.predict(val_inputs)
print(classification_report(val_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.51      0.46      1578
           B       0.40      0.23      0.29      1486
           C       0.50      0.60      0.55      1576
           D       0.67      0.65      0.66      1814

    accuracy                           0.51      6454
   macro avg       0.50      0.50      0.49      6454
weighted avg       0.50      0.51      0.50      6454



**Спостереження**
Я обрала метрику f1-score. Великої різниці між різними наборами даних немає. Я думаю це тому, що ремасплінг не побудував точки, які значно повпливали на класифікатор